In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import optuna


# Pandas setting to display more dataset rows and columns
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Data import**

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/train.csv", low_memory=False)#, nrows=10000)
test = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/test.csv", low_memory=False)#, nrows=10000)
train.info(memory_usage="deep")

In [ ]:
test.info(memory_usage="deep")

# **EDA**

In [ ]:
# Colors to be used for plots
colors = ["lightcoral", "sandybrown", "darkorange", "mediumseagreen",
          "lightseagreen", "cornflowerblue", "mediumpurple", "palevioletred",
          "lightskyblue", "sandybrown", "yellowgreen", "indianred",
          "lightsteelblue", "mediumorchid", "deepskyblue"]

In [ ]:
train.head()

In [ ]:
train.columns.values

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
pie = ax.pie([len(train), len(test)],
             labels=["Train dataset", "Test dataset"],
             colors=["salmon", "teal"],
             textprops={"fontsize": 15},
             autopct='%1.1f%%')
ax.axis("equal")
ax.set_title("Dataset length comparison", fontsize=18)
fig.set_facecolor('white')
plt.show();

In [ ]:
train.describe().T

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

bars = ax.bar(train["claim"].value_counts().index,
              train["claim"].value_counts().values,
              color=colors,
              edgecolor="black",
              width=0.4)
ax.set_title("Claim (target) values distribution", fontsize=20, pad=15)
ax.set_ylabel("Amount of values", fontsize=14, labelpad=15)
ax.set_xlabel("Claim (target) value", fontsize=14, labelpad=10)
ax.set_xticks(train["claim"].value_counts().index)
ax.tick_params(axis="both", labelsize=14)
ax.bar_label(bars, [f"{x:2.2f}%" for x in train["claim"].value_counts().values/(len(train)/100)],
                 padding=5, fontsize=15)
ax.bar_label(bars, [f"{x:2d}" for x in train["claim"].value_counts().values],
                 padding=-30, fontsize=15)
ax.margins(0.2, 0.12)
ax.grid(axis="y")

plt.show();

The target value classes are balanced which is good.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))

bars = ax.bar(train.isna().sum().index,
              train.isna().sum().values,
              color="lightskyblue",
              edgecolor="black",
              width=0.7)
ax.set_title("Missing feature values distribution in the train dataset", fontsize=20, pad=15)
ax.set_ylabel("Missing values", fontsize=14, labelpad=15)
ax.set_xlabel("Feature", fontsize=14, labelpad=10)
ax.set_xticks([x if i%2==0 else "" for i, x in enumerate(train.columns.values)])
ax.tick_params(axis="x", rotation=90, labelsize=8)
ax.margins(0.005, 0.12)
ax.grid(axis="y")

plt.show();

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))

bars = ax.bar(test.isna().sum().index,
              test.isna().sum().values,
              color="lightsteelblue",
              edgecolor="black",
              width=0.7)
ax.set_title("Missing feature values distributionin in the test dataset", fontsize=20, pad=15)
ax.set_ylabel("Missing values", fontsize=14, labelpad=15)
ax.set_xlabel("Feature", fontsize=14, labelpad=10)
ax.set_xticks([x if i%2==0 else "" for i, x in enumerate(test.columns.values)])
ax.tick_params(axis="x", rotation=90, labelsize=8)
ax.margins(0.005, 0.12)
ax.grid(axis="y")

plt.show();

As you can see, both train and test datasets have missing values in every feature excepth for "id" and "claim". We should take care with them.

Let's check feature values distribution in the both datasets.

In [ ]:
df = pd.concat([train.drop(["id", "claim"], axis=1), test.drop("id", axis=1)], axis=0)
columns = df.columns.values

cols = 4
rows = len(columns) // cols + 1

fig, axs = plt.subplots(ncols=cols, nrows=rows, figsize=(16,130), sharex=False)

plt.subplots_adjust(hspace = 0.3)
i=0

for r in np.arange(0, rows, 1):
    for c in np.arange(0, cols, 1):
        if i >= len(columns):
            axs[r, c].set_visible(False)
        else:
            hist1 = axs[r, c].hist(train[columns[i]].values,
                                   range=(df[columns[i]].min(),
                                          df[columns[i]].max()),
                                   bins=40,
                                   color="deepskyblue",
                                   edgecolor="black",
                                   alpha=0.7,
                                   label="Train Dataset")
            hist2 = axs[r, c].hist(test[columns[i]].values,
                                   range=(df[columns[i]].min(),
                                          df[columns[i]].max()),
                                   bins=40,
                                   color="palevioletred",
                                   edgecolor="black",
                                   alpha=0.7,
                                   label="Test Dataset")
            axs[r, c].set_title(columns[i], fontsize=12, pad=5)
            axs[r, c].set_yticks(axs[r, c].get_yticks())
            axs[r, c].set_yticklabels([str(int(i/1000))+"k" for i in axs[r, c].get_yticks()])
            axs[r, c].tick_params(axis="y", labelsize=10)
            axs[r, c].tick_params(axis="x", labelsize=10)
            axs[r, c].grid(axis="y")
            axs[r, c].legend(fontsize=13)
                                  
        i+=1
#plt.suptitle("Feature values distribution in both datasets", y=0.99)
plt.show();

As you can see, the datasets are well balanced. So target distribution should probably be the same for test predictions.

In [ ]:
print("Features with the leas amount of unique values:")
train.drop(["id", "claim"], axis=1).nunique().sort_values().head(5)

There are no categorical features in the dataset.

Let's look at feature correlation.

In [ ]:
# Plot dataframe
df = train.drop("id", axis=1).corr().round(5)

# Mask to hide upper-right part of plot as it is a duplicate
mask = np.zeros_like(df)
mask[np.triu_indices_from(mask)] = True

# Making a plot
plt.figure(figsize=(16,16))
ax = sns.heatmap(df, annot=False, mask=mask, cmap="RdBu", annot_kws={"weight": "bold", "fontsize":13})
ax.set_title("Feature correlation heatmap", fontsize=17)
plt.setp(ax.get_xticklabels(), rotation=90, ha="right",
         rotation_mode="anchor", weight="normal")
plt.setp(ax.get_yticklabels(), weight="normal",
         rotation_mode="anchor", rotation=0, ha="right")
plt.show();

There is very weak linear correlation between the features. There are some features with relatively low correlation with target value even comparing with other features:

In [ ]:
df[(df["claim"]>-0.001) & (df["claim"]<0.001)]["claim"]

Probably worth a try to drop them and check if it improves the result.

# **Data preprocessing**

In [ ]:
features = [x for x in train.columns.values if x[0]=="f"]

The idea of adding a new feature below is taken from [this notebook](https://www.kaggle.com/hiro5299834/tps-sep-2021-single-lgbm) by [BIZEN](https://www.kaggle.com/hiro5299834).

In [ ]:
# Counting amount of missing values in each row and adding it as a new feature
train['n_missing'] = train[features].isna().sum(axis=1)
test['n_missing'] = test[features].isna().sum(axis=1)

In [ ]:
# Filling missing values with mean of each column
imputer = SimpleImputer(strategy="median")
for col in features:
    train[col] = imputer.fit_transform(np.array(train[col]).reshape(-1,1))
    test[col] = imputer.transform(np.array(test[col]).reshape(-1,1))

In [ ]:
# Scaling all values to [0,1] range
s_scaler = StandardScaler()
m_scaler = MinMaxScaler()
for col in features:
    train[col] = s_scaler.fit_transform(np.array(train[col]).reshape(-1,1))
    test[col] = s_scaler.transform(np.array(test[col]).reshape(-1,1))
    train[col] = m_scaler.fit_transform(np.array(train[col]).reshape(-1,1))
    test[col] = m_scaler.transform(np.array(test[col]).reshape(-1,1))

In [ ]:
X = train.drop(["id", "claim"], axis=1)
X_test = test.drop("id", axis=1)
y = train["claim"]

# **Hyperparameters optimization**

Hyperparameters used in this notebook were optimized using Optuna. The code used or that is shown below. They are commented in order to save runtime as optimization has been already done.

In [ ]:
# def train_model_optuna(trial, X_train, X_valid, y_train, y_valid):
#     """
#     A function to train a model using different hyperparamerters combinations provided by Optuna. 
#     Loss of validation data predictions is returned to estimate hyperparameters effectiveness.
#     """
#     preds = 0
    
        
#     #A set of hyperparameters to optimize by optuna
#     xgb_params = {
#                  "objective": trial.suggest_categorical('objective', ["binary:logistic"]),
#                  "use_label_encoder": trial.suggest_categorical('use_label_encoder', [False]),
#                  "n_estimators": trial.suggest_categorical('n_estimators', [40000]),
#                  "learning_rate": trial.suggest_loguniform('learning_rate', 0.1, 1.0),
#                  "subsample": trial.suggest_float('subsample', 0.1, 1, step=0.01),
#                  "colsample_bytree": trial.suggest_float('colsample_bytree', 0.05, 1, step=0.01),
#                  "max_depth": trial.suggest_int("max_depth", 1, 8),
#                  "booster": trial.suggest_categorical('booster', ["gbtree"]),
#                  "gamma": trial.suggest_float('gamma', 0, 100, step=0.1),
# #                  "tree_method": trial.suggest_categorical('tree_method', ["gpu_hist"]),
#                  "reg_lambda": trial.suggest_loguniform('reg_lambda', 0.1, 100),
#                  "reg_alpha": trial.suggest_loguniform('reg_alpha', 0.1, 100),
#                  "random_state": trial.suggest_categorical('random_state', [42]),
#                  "n_jobs": trial.suggest_categorical('n_jobs', [4]),
#                  "min_child_weight": trial.suggest_categorical("min_child_weight", [256]),
#                     }



#     # Model loading and training
#     model = XGBClassifier(**xgb_params)
#     model.fit(X_train, y_train,
#               eval_set=[(X_train, y_train), (X_valid, y_valid)],
#               eval_metric="auc",
#               early_stopping_rounds=100,
#               verbose=False)
    
#     print(f"Number of boosting rounds: {model.best_iteration}")
#     oof = model.predict_proba(X_valid)[:, 1]
    
#     return roc_auc_score(y_valid, oof)

In [ ]:
# %%time
# # Splitting data into train and valid folds using target bins for stratification
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# # Setting optuna verbosity to show only warning messages
# # If the line is uncommeted each iteration results will be shown
# # optuna.logging.set_verbosity(optuna.logging.WARNING)

# time_limit = 3600 * 4

# study = optuna.create_study(direction='maximize')
# study.optimize(lambda trial: train_model_optuna(trial, X_train, X_valid,
#                                                     y_train, y_valid),
# #                n_trials = 2
#                timeout=time_limit
#               )

# # Showing optimization results
# print('Number of finished trials:', len(study.trials))
# print('Best trial parameters:', study.best_trial.params)
# print('Best score:', study.best_value)

# **Model training**

In [ ]:
# Model hyperparameters
xgb_params = {'objective': 'binary:logistic',
              'use_label_encoder': False,
              'n_estimators': 40000,
              'learning_rate': 0.35,
              'subsample': 0.88,
              'colsample_bytree': 0.51,
              'max_depth': 1,
              'booster': 'gbtree',
              'gamma': 0.2,
#               'tree_method': 'gpu_hist',
              'reg_lambda': 0.7497270276048157,
              'reg_alpha': 52.11267615877387, 
              'random_state': 42,
              'n_jobs': 4,
              'min_child_weight': 256}

In [ ]:
%%time
# Setting up fold parameters
splits = 5
skf = StratifiedKFold(n_splits=splits, shuffle=True, random_state=42)

# Creating an array of zeros for storing "out of fold" predictions
oof_preds = np.zeros((X.shape[0],))
preds = 0
model_fi = 0
total_mean_auc = 0

# Generating folds and making training and prediction for each of 10 folds
for num, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
    X_train, X_valid = X.loc[train_idx], X.loc[valid_idx]
    y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]
    
    model = XGBClassifier(**xgb_params)
    model.fit(X_train, y_train,
              verbose=False,
              # These three parameters will stop training before a model starts overfitting 
              eval_set=[(X_train, y_train), (X_valid, y_valid)],
              eval_metric="auc",
              early_stopping_rounds=300,
              )
    
    # Getting mean test data predictions (i.e. devided by number of splits)
    preds += model.predict_proba(X_test)[:, 1] / splits
    
    # Getting mean feature importances (i.e. devided by number of splits)
    model_fi += model.feature_importances_ / splits
    
    # Getting validation data predictions. Each fold model makes predictions on an unseen data.
    # So in the end it will be completely filled with unseen data predictions.
    # It will be used to evaluate hyperparameters performance only.
    oof_preds[valid_idx] = model.predict_proba(X_valid)[:, 1]
    
    # Getting score for a fold model
    fold_auc = roc_auc_score(y_valid, oof_preds[valid_idx])
    print(f"Fold {num} ROC AUC: {fold_auc}")

    # Getting mean score of all fold models (i.e. devided by number of splits)
    total_mean_auc += fold_auc / splits
    
print(f"\nOverall ROC AUC: {total_mean_auc}")

# **Feature importances**

In [ ]:
# Creating a dataframe to be used for plotting
df = pd.DataFrame()
df["Feature"] = X.columns
# Extracting feature importances from the trained model
df["Importance"] = model_fi / model_fi.sum()
# Sorting the dataframe by feature importance
df.sort_values("Importance", axis=0, ascending=False, inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(13, 35))
bars = ax.barh(df["Feature"], df["Importance"], height=0.4,
               color="mediumorchid", edgecolor="black")
ax.set_title("Feature importances", fontsize=30, pad=15)
ax.set_ylabel("Feature name", fontsize=20, labelpad=15)
ax.set_xlabel("Feature importance", fontsize=20, labelpad=15)
ax.set_yticks(df["Feature"])
ax.set_yticklabels(df["Feature"], fontsize=15)
ax.tick_params(axis="x", labelsize=15)
ax.grid(axis="x")
# Adding labels on top
ax2 = ax.secondary_xaxis('top')
ax2.set_xlabel("Feature importance", fontsize=20, labelpad=15)
ax2.tick_params(axis="x", labelsize=15)

# Inverting y axis direction so the values are decreasing
plt.gca().invert_yaxis()

# **Predictions submission**

In [ ]:
predictions = pd.DataFrame()
predictions["id"] = test["id"]
predictions["claim"] = preds

predictions.to_csv('submission.csv', index=False, header=predictions.columns)
predictions.head()